# dina-py Library Tutorial

This notebook demonstrates how to use the dina-py Python library for interacting with DINA.

# 🛠 Installation Guide

This notebook walks you through setting up your environment and installing the `dinapy` library for use in Jupyter Notebooks.

# Manual Setup (outside the notebook)

## ✅ Step 1: Make sure to have Python 3.10 or newer

## ✅ Step 2: Create a Virtual Environment

Run the following command in your TERMINAL (NOT Jupyter notebook):

In [ ]:
python -m venv env


### 🔹 Activate the Environment

- **Windows:**


In [ ]:
.\env\Scripts\activate

- **macOS/Linux:**

In [ ]:
source env/bin/activate

## ✅ Step 3: Install `ipykernel`

ipykernel is needed to run code in the jupyter notebook

Run the following in your terminal:

In [ ]:
pip install ipykernel

## ✅ Step 4: Install `dinapy`

### From Git (branch) — works in Jupyter or terminal
If on Jupyter notebook, just run the cell below:

In [ ]:
# Start clean: remove any previously installed dinapy
%pip uninstall -y dinapy

# Equivalent to your original, but more resilient parsing for extras:
%pip install "dinapy[notebook] @ git+https://github.com/AAFC-BICoE/dina-py.git@Support_36999_jupyter_notebook_test"

## ✅ Step 4: Launch Jupyter Notebook

You can now launch Jupyter Notebook server with the following command in your TERMINAL:

In [ ]:
jupyter notebook

## Import the Library

Let's import the dina-py library here and check if it's working.

In [ ]:
# Import dina-py
from importlib.metadata import version, PackageNotFoundError

try:
    import dinapy
    print(f"dina-py successfully imported!")

    try:
        print("Version:", version("dinapy"))
    except PackageNotFoundError:
        print("Version: Unknown (package not installed)")
except ImportError as e:
    print(f"Error importing dina-py: {e}")
    print("Please install it using: pip install dina-py")

## Configuration

Set up .env variables with ipywidgets


In [ ]:
%pip install python-dotenv ipywidgets

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display

# Create input widgets
username = widgets.Text(value='', placeholder='dina-admin', description='Username:')
password = widgets.Password(value='', placeholder='dina-admin', description='Password:')
domain = widgets.Text(value='', placeholder='dina-dev2.biodiversity.agr.gc.ca', description='Keycloak Domain:', layout=widgets.Layout(width='600px'))

# Output widget
output = widgets.Output()

# Button widget
set_button = widgets.Button(
    description="Set and Save Environment Variables",
    button_style='success',
    layout=widgets.Layout(width='auto', height='40px')
)

# Function to run on button click
def on_button_click(b):
    keycloak_url = f"https://{domain.value.strip()}"

    os.environ["KEYCLOAK_USERNAME"] = username.value
    os.environ["KEYCLOAK_PASSWORD"] = password.value
    os.environ["DINA_API_BASE_URL"] = "dina.local"
    os.environ["KEYCLOAK_URL"] = keycloak_url
    os.environ["CLIENT_ID"] = "dina-public"
    os.environ["REALM_NAME"] = "dina"
    os.environ["SECURE"] = "true"

    with open(".env", "w") as f:
        f.write(f"KEYCLOAK_USERNAME={username.value}\n")
        f.write(f"KEYCLOAK_PASSWORD={password.value}\n")
        f.write(f"DINA_API_BASE_URL=dina.local\n")
        f.write(f"KEYCLOAK_URL={keycloak_url}\n")
        f.write(f"CLIENT_ID=dina-public\n")
        f.write(f"REALM_NAME=dina\n")
        f.write(f"SECURE=true\n")

    with output:
        output.clear_output()
        print("✅ Environment variables set and saved to .env file.")

# Bind button to function
set_button.on_click(on_button_click)

# Display widgets and button
display(username, password, domain, set_button, output)

### Jupyter Cell to Install SSL Certificate from KEYCLOAK_URL


In [ ]:
import os
import subprocess
import certifi
from urllib.parse import urlparse

# Ensure KEYCLOAK_URL is set
keycloak_url = os.environ.get("KEYCLOAK_URL")
if not keycloak_url:
    raise ValueError("KEYCLOAK_URL environment variable is not set.")

# Extract hostname from URL
parsed_url = urlparse(keycloak_url)
hostname = parsed_url.hostname
if not hostname:
    raise ValueError("Invalid KEYCLOAK_URL format.")

# Define file paths
certs_txt = "certs.txt"
combined_cert = "combined-cert.crt"
certifi_path = certifi.where()

# Run openssl to get the certificate and extract it using awk
subprocess.run(f"openssl s_client -connect {hostname}:443 -showcerts < /dev/null > {certs_txt}", shell=True, check=True)
subprocess.run(f"awk '/-----BEGIN CERTIFICATE-----/,/-----END CERTIFICATE-----/' {certs_txt} > {combined_cert}", shell=True, check=True)

# Append the certificate to certifi's CA bundle
with open(certifi_path, "ab") as ca_bundle, open(combined_cert, "rb") as new_cert:
    ca_bundle.write(b"\n")
    ca_bundle.write(new_cert.read())

print(f"✅ Certificate from {hostname} added to certifi store at {certifi_path}.")

## Simple test using .env

In [ ]:
from dinapy.apis.collectionapi.managedattributesapi import ManagedAttributeAPI
import requests
 
dina_managed_attribute_api = ManagedAttributeAPI()
 
print(requests.get("https://dina.local").status_code)
response = dina_managed_attribute_api.get_entity("019a020c-2864-72d8-90f9-82e964422620")
print("Fetched Entity:", response.json()["data"])

## Example: Create a New Record

Create a new record in DINA (adjust according to your data model).

In [ ]:
from dinapy.apis.collectionapi.managedattributesapi import ManagedAttributeAPI
from dinapy.entities.ManagedAttribute import ManagedAttributesDTOBuilder,ManagedAttributeAttributesDTOBuilder
from dinapy.schemas.managedattributeschema import ManagedAttributesSchema

def main():
 dina_managed_attribute_api = ManagedAttributeAPI()
 for i in range(1, 16):
  # Build the attribute DTO
  managed_attribute_attributes = (
   ManagedAttributeAttributesDTOBuilder()
   .name(f"test attributes{i}")
   .vocabularyElementType("STRING")
   .group("aafc")
   .managedAttributeComponent("MATERIAL_SAMPLE")
   .build()
  )

  # Wrap it in the managed attribute DTO
  managed_attribute = ManagedAttributesDTOBuilder().attributes(managed_attribute_attributes).build()

  # Serialize using schema
  managed_attribute_schema = ManagedAttributesSchema()
  serialized_managed_attribute = managed_attribute_schema.dump(managed_attribute)

  # Send to API
  response = dina_managed_attribute_api.create_entity(serialized_managed_attribute)

  # Print the resulting IDs
  print(response.json()['data']['id'])

if __name__ == '__main__':
 main()

## Sample Helper Functions

Here are some helper functions that might be useful when working with dina-py.

In [ ]:
import pandas as pd
import json

def display_records_as_dataframe(records):
    """Convert DINA records to a pandas DataFrame for easier viewing"""
    if not records:
        print("No records to display")
        return None
    
    df = pd.DataFrame(records)
    return df

def pretty_print_json(data):
    """Pretty print JSON data"""
    print(json.dumps(data, indent=2))

print("Helper functions loaded!")

## Your Custom Code

Use the cells below to write your own code using dina-py.

In [ ]:
# Your code here


## Resources

- [DINA Documentation](https://dina-project.github.io/)
- [dina-py GitHub Repository](https://github.com/DINA-Web/dina-py)
- [DINA API Documentation](https://api.dina-project.net/)

## Notes

- Make sure to replace placeholder values with your actual DINA instance credentials
- The exact methods and parameters may vary depending on the version of dina-py you're using
- Always refer to the official documentation for the most up-to-date information
- Be careful with credentials - consider using environment variables for sensitive information